# FnGuide Crawling

In [1]:
#!/usr/bin/env python3
"""
리팩토링된 동적 테이블 크롤러
- 함수 분리로 가독성 향상
- 클래스 기반 구조화
- 에러 처리 개선
- 디버그 모드 분리
"""

import json
import logging
import pandas as pd
import requests
from bs4 import BeautifulSoup, Tag
from io import StringIO
from typing import Dict, List, Tuple, Optional, Any

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## FnGuide Url 목록

- [메인](https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{stock})
- [기업개요](https://comp.fnguide.com/SVO2/ASP/SVD_Corp.asp?pGB=1&gicode=A{stock})
- [재무제표](https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock})
- [재무비율](https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{stock})
- [투자지표](https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{stock})
- [컨센서스](https://comp.fnguide.com/SVO2/ASP/SVD_Consensus.asp?pGB=1&gicode=A{stock})
- [지분분석](https://comp.fnguide.com/SVO2/ASP/SVD_shareanalysis.asp?pGB=1&gicode=A{stock})
- [업종분석](https://comp.fnguide.com/SVO2/ASP/SVD_ujanal.asp?pGB=1&gicode=A{stock})
- [경쟁사비교](https://comp.fnguide.com/SVO2/ASP/SVD_Comparison.asp?pGB=1&gicode=A{stock})
- [거래소공시](https://comp.fnguide.com/SVO2/ASP/SVD_Disclosure.asp?pGB=1&gicode=A{stock})
- [금감원공시](https://comp.fnguide.com/SVO2/ASP/SVD_Dart.asp?pGB=1&gicode=A{stock})

- [삼성전자(A005930) | Snapshot | 기업정보 | Company Guide](https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930)
- [삼성전자(A005930) | 재무제표 | 기업정보 | Company Guide, 메뉴 포함](https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701)
- [삼성전자(A005930) | 재무제표 | 기업정보 | Company Guide, 메뉴 제외](https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930)

```
FnGuideCrawler
├── FnGuideMain
│   ├── fetch (메인 URL으로 HTML 정보 가져오기)
│   ├── parse (메인 HTML 정보 파싱)
│   ├── check_gcp (GCP에서 메인 Caching 정보 확인)
│   └── save_gcp (GCP에서 메인 Caching 정보 저장)
├── FnGuideFinance
│   ├── fetch (재무제표 URL으로 HTML 정보 가져오기)
│   ├── parse (재무제표 HTML 정보 파싱)
│   ├── check_gcp (GCP에서 재무제표 Caching 정보 확인)
│   ├── save_gcp (GCP에서 재무제표 Caching 정보 확인)
│   ├── TableFinder
│   ├── HeaderExtractor
│   └── BodyExtractor
├── FnGuideCompany
├── FnGuideFinanceRatio
└── ...
```

In [2]:
stock = "005930" # 삼성전자

In [3]:
#Snapshot
main_url = "https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{stock}"
#기업개요
company_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Corp.asp?pGB=1&gicode=A{stock}"
#재무제표
#dynamic_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"
finance_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock}"
#재무비율
finance_ratio_url = "https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{stock}"
#투자지표
invest_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{stock}"
#컨센서스
consensus_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Consensus.asp?pGB=1&gicode=A{stock}"
#지분분석
share_analysis_url = "https://comp.fnguide.com/SVO2/ASP/SVD_shareanalysis.asp?pGB=1&gicode=A{stock}"
#업종분석
industry_analysis_url = "https://comp.fnguide.com/SVO2/ASP/SVD_ujanal.asp?pGB=1&gicode=A{stock}"
#경쟁사비교
comparison_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Comparison.asp?pGB=1&gicode=A{stock}"
#거래소공시
disclosure_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Disclosure.asp?pGB=1&gicode=A{stock}"
#금감원공시
dart_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Dart.asp?pGB=1&gicode=A{stock}"

In [4]:
urls = {
    "메인": "https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{stock}",
    "기업개요": "https://comp.fnguide.com/SVO2/ASP/SVD_Corp.asp?pGB=1&gicode=A{stock}",
    "재무제표": "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock}",
    "재무비율": "https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{stock}",
    "투자지표": "https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{stock}",
    "컨센서스": "https://comp.fnguide.com/SVO2/ASP/SVD_Consensus.asp?pGB=1&gicode=A{stock}",
    "지분분석": "https://comp.fnguide.com/SVO2/ASP/SVD_shareanalysis.asp?pGB=1&gicode=A{stock}",
    "업종분석": "https://comp.fnguide.com/SVO2/ASP/SVD_ujanal.asp?pGB=1&gicode=A{stock}",
    "경쟁사비교": "https://comp.fnguide.com/SVO2/ASP/SVD_Comparison.asp?pGB=1&gicode=A{stock}",
    "거래소공시": "https://comp.fnguide.com/SVO2/ASP/SVD_Disclosure.asp?pGB=1&gicode=A{stock}",
    "금감원공시": "https://comp.fnguide.com/SVO2/ASP/SVD_Dart.asp?pGB=1&gicode=A{stock}"
}

## 재무제표 정보

- 재무제표 테이블 분석
- 테이블 검색 클래스
- 테이블 헤더 추출 클래스
- 테이블 바디 추출 클래스

- 포괄손익계산서 [연간 | 분기]
  - 주요 재무항목, 성장성 지표
  - IFRS(연결)
- 재무상태표 [연간 | 분기]
  - 주요 재무항목, 안정성 지표
  - IFRS(연결)
- 현금흐름표 [연간 | 분기]
  - Invested Capital, Free Cash Flow
  - IFRS(연결)

In [1]:
class TableFinder:
    """테이블 검색 클래스"""
    
    @staticmethod
    def find_by_title(soup: BeautifulSoup, title: str) -> Optional[Tag]:
        """제목으로 테이블 찾기"""
        for table in soup.find_all("table"):
            if title in table.get_text():
                logger.info(f"'{title}' 테이블 발견")
                return table
        
        logger.warning(f"'{title}' 테이블을 찾을 수 없음")
        return None


class HeaderExtractor:
    """테이블 헤더 추출 클래스"""
    
    @staticmethod
    def extract_index_list(thead: Tag) -> List[str]:
        """
        thead에서 인덱스 리스트 추출
        
        Args:
            thead: thead 태그
        
        Returns:
            인덱스 리스트 (날짜/기간 데이터)
        """
        thead_rows = thead.find_all("tr")
        index_rows: List[List[str]] = []
        
        for tr in thead_rows:
            row_headers = HeaderExtractor._extract_row_headers(tr)
            if row_headers:
                index_rows.append(row_headers)
        
        # 가장 긴 행을 선택 (가장 상세한 헤더)
        index_list = max(index_rows, key=len) if index_rows else []
        
        total_headers = sum(len(tr.find_all("th")) for tr in thead_rows)
        logger.info(f"기간 데이터 (헤더 {total_headers}개 중 {len(index_list)}개): {index_list}")
        
        return index_list
    
    @staticmethod
    def _extract_row_headers(tr: Tag) -> List[str]:
        """
        단일 행에서 헤더 추출 (colspan 처리)
        
        Args:
            tr: tr 태그
        
        Returns:
            헤더 리스트
        """
        row_headers = []
        ths = tr.find_all("th", recursive=False)
        
        for col_idx, th in enumerate(ths):
            text = th.get_text(strip=True)
            if not text:
                continue
            
            # 첫 번째 th는 행 구분자로 스킵 (2개 이상일 때만)
            if col_idx == 0 and len(ths) > 1:
                continue
            
            # colspan 처리
            colspan = HeaderExtractor._get_colspan(th)
            row_headers.extend([text] * colspan)
        
        return row_headers
    
    @staticmethod
    def _get_colspan(th: Tag) -> int:
        """colspan 속성 추출"""
        colspan_attr = th.get("colspan")
        try:
            return int(colspan_attr) if colspan_attr else 1
        except ValueError:
            return 1


class BodyExtractor:
    """테이블 바디 추출 클래스"""
    
    def __init__(self, debug: bool = False):
        self.debug = debug
        self.last_span_text: Optional[str] = None
    
    def extract(
        self,
        tbody: Tag,
        index_list: List[str]
    ) -> Dict[Tuple[str, str], List[str]]:
        """
        tbody에서 데이터 딕셔너리 추출
        
        Args:
            tbody: tbody 태그
            index_list: 헤더 인덱스 리스트
        
        Returns:
            {(카테고리, 항목): [값들]} 형태의 딕셔너리
        """
        data_dict = {}
        tbody_trs = tbody.find_all("tr", recursive=False)
        
        logger.info(f"발견된 행 수: {len(tbody_trs)}")
        
        if self.debug and tbody_trs:
            logger.debug(f"첫 행 HTML: {str(tbody_trs[0])[:500]}")
        
        processed_count = 0
        
        for idx, tr in enumerate(tbody_trs):
            self._log_progress(idx, len(tbody_trs))
            
            try:
                # 행 데이터 추출
                column_tuple, values = self._extract_row_data(tr, idx)
                
                if not column_tuple or not values:
                    continue
                
                # 헤더와 값 개수 검증
                if len(values) == len(index_list):
                    data_dict[column_tuple] = values
                    processed_count += 1
                    
                    if self.debug and idx < 3:
                        logger.debug(f"행 {idx} 성공!")
                else:
                    if self.debug and idx < 5:
                        logger.warning(
                            f"행 {idx} 길이 불일치: "
                            f"헤더={len(index_list)}, 값={len(values)} (스킵)"
                        )
            
            except Exception as e:
                if self.debug and idx < 5:
                    logger.error(f"행 {idx} 처리 중 에러: {e}")
                continue
        
        logger.info(f"처리 완료: {processed_count}/{len(tbody_trs)} 행")
        
        return data_dict
    
    def _extract_row_data(
        self,
        tr: Tag,
        idx: int
    ) -> Tuple[Optional[Tuple[str, str]], List[str]]:
        """
        단일 행에서 컬럼명과 값 추출
        
        Args:
            tr: tr 태그
            idx: 행 인덱스
        
        Returns:
            (컬럼명_튜플, 값_리스트)
        """
        # th 찾기
        th = tr.find("th")
        if not th:
            if self.debug and idx < 3:
                logger.debug(f"행 {idx}: th 없음")
            return None, []
        
        # 컬럼명 추출
        column_tuple = self._extract_column_name(th, idx)
        
        # td 값들 추출
        tds = tr.find_all("td", recursive=False)
        values = [td.get_text(strip=True) for td in tds]
        
        if self.debug and idx < 3:
            logger.debug(
                f"행 {idx}: column={column_tuple}, "
                f"td 개수={len(tds)}, 값={values[:3] if values else '없음'}"
            )
        
        return column_tuple, values
    
    def _extract_column_name(self, th: Tag, idx: int) -> Tuple[str, str]:
        """
        th에서 컬럼명 튜플 추출 (span 고려)
        
        Args:
            th: th 태그
            idx: 행 인덱스
        
        Returns:
            (카테고리, 항목) 튜플
        """
        span = th.find("span")
        th_text = th.get_text(strip=True)
        
        # span이 있는 경우: 새로운 상위 카테고리
        if span:
            span_text = span.get_text(strip=True)
            self.last_span_text = span_text
            column_tuple = (span_text, th_text)
            
            if self.debug and idx < 3:
                logger.debug(f"행 {idx} (span): {column_tuple}")
        
        # span 없는 경우: 이전 카테고리 사용
        else:
            if self.last_span_text:
                column_tuple = (self.last_span_text, th_text)
            else:
                column_tuple = (th_text, "")
            
            if self.debug and idx < 3:
                logger.debug(f"행 {idx} (no span): {column_tuple}")
        
        return column_tuple
    
    @staticmethod
    def _log_progress(idx: int, total: int):
        """
        진행상황 로깅 (20개마다)
        
        Args:
            idx (int): 로그 번호
            total (int): 로그 최대값
        """
        if idx > 0 and idx % 20 == 0:
            logger.info(f"처리 중: {idx}/{total} 행")

NameError: name 'BeautifulSoup' is not defined

In [6]:
# =============================================================================
# 재무제표 분석 클래스
# =============================================================================

class FnGuideFinance:
    """재무제표 테이블 분석"""

    finance_table_titles = ["포괄손익계산서", "재무상태표", "현금흐름표"]
    
    def __init__(self, debug: bool = False):
        """
        Args:
            debug: 디버그 모드
        """
        self.debug = debug
        self.table_finder = TableFinder()
        self.header_extractor = HeaderExtractor()

    def fetch(
        self,
        stock: str,
        url: str) -> str:
        """
        URL 정보를 가져오기

        Args:
            stock (str): 종목 코드
            url (str): URL 템플릿 ('{stock}' 포함)

        Returns:
            str: 웹페이지 HTML 텍스트
        """
        # 페이지 가져오기
        self.stock = stock
        
        url = url.format(stock=stock, timeout=30)
        logger.info(f"Main URL: {url}")
        
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            logger.error(f"페이지 요청 실패: {e}")
            return None

    def parse(
        self,
        html_text: str,
        *,
        stock: str | None = None,
    ) -> Dict[str, List[Dict]]:
        """
        재무제표 테이블 수집 (requests + BeautifulSoup 사용)

        재무제표 3종(포괄손익계산서, 재무상태표, 현금흐름표)을
        requests와 BeautifulSoup으로 크롤링하여 멀티인덱스 DataFrame으로 구조화

        Args:
            html_text (str): 웹페이지 HTML 텍스트
            stock (str): 종목 코드

        Returns:
            {테이블명: 레코드_리스트} 딕셔너리
        """

        soup = BeautifulSoup(html_text, "html.parser")

        # 테이블 파싱
        result_dict = {}
        
        for title in self.finance_table_titles:
            try:
                df = self.parse_table(soup, title)
                
                if df is not None:
                    # DataFrame을 레코드로 변환
                    result_dict[title] = self._dataframe_to_records(df)
                    #print(result_dict[title])
                else:
                    result_dict[title] = []
            
            except Exception as e:
                logger.error(f"{title} 수집 실패: {e}")
                result_dict[title] = []
        
        return result_dict
        
    
    def parse_table(
        self,
        soup: BeautifulSoup,
        title: str
    ) -> Optional[pd.DataFrame]:
        """
        단일 테이블 파싱
        
        Args:
            soup: BeautifulSoup 객체
            title: 테이블 제목
        
        Returns:
            DataFrame 또는 None
        """
        logger.info(f"\n{title} 데이터 수집 중...")
        
        # 1. 테이블 찾기
        table = self.table_finder.find_by_title(soup, title)
        if not table:
            return None
        
        # 2. thead 추출
        thead = table.find("thead")
        if not thead:
            logger.warning("thead를 찾을 수 없음")
            return None
        
        # 3. 헤더 인덱스 추출
        index_list = self.header_extractor.extract_index_list(thead)
        if not index_list:
            logger.warning("인덱스 리스트가 비어있음")
            return None
        
        # 4. tbody 추출
        tbody = table.find("tbody")
        if not tbody:
            logger.warning("tbody를 찾을 수 없음")
            return None
        
        # 5. 데이터 딕셔너리 추출
        body_extractor = BodyExtractor(debug=self.debug)
        data_dict = body_extractor.extract(tbody, index_list)
        
        if not data_dict:
            logger.warning("데이터가 비어있음")
            return None
        
        # 6. DataFrame 생성
        df = self._create_dataframe(data_dict, index_list)
        
        logger.info(f"완료! DataFrame shape: {df.shape}")
        
        return df
        
    def _dataframe_to_records(self, frame: pd.DataFrame) -> list[dict[str, Any]]:
        """
        DataFrame을 JSON 직렬화를 위한 레코드 리스트로 변환한다.
    
        멀티인덱스 컬럼은 " / "로 결합된 단일 키로 변환하고,
        인덱스(기간)는 'period' 필드로 포함한다.
        """
        if frame.empty:
            return []
    
        flattened_columns = [self._flatten_column_key(col) for col in frame.columns]
        records: list[dict[str, Any]] = []
    
        for index_label, row in frame.iterrows():
            record: dict[str, Any] = {"period": str(index_label)}
            for key, value in zip(flattened_columns, row.tolist()):
                if key in record:
                    suffix = 2
                    new_key = f"{key}_{suffix}"
                    while new_key in record:
                        suffix += 1
                        new_key = f"{key}_{suffix}"
                    record[new_key] = value
                else:
                    record[key] = value
            records.append(record)
    
        return records

    @staticmethod
    def _create_dataframe(
        data_dict: Dict[Tuple[str, str], List[str]],
        index_list: List[str]
    ) -> pd.DataFrame:
        """
        데이터 딕셔너리에서 DataFrame 생성
        
        Args:
            data_dict: {(카테고리, 항목): [값들]} 딕셔너리
            index_list: 인덱스 리스트
        
        Returns:
            멀티인덱스 컬럼을 가진 DataFrame
        """
        df = pd.DataFrame(data_dict, index=index_list)
        
        # 멀티인덱스로 컬럼 변환
        df.columns = pd.MultiIndex.from_tuples(df.columns)
        
        return df

    @staticmethod
    def _flatten_column_key(column: Any) -> str:
        """
        멀티인덱스 컬럼 키를 JSON 직렬화가 가능한 단일 문자열로 변환한다.
        """
        if isinstance(column, tuple):
            parts = [str(part).strip() for part in column if part not in (None, "")]
            key = " / ".join(parts)
        elif column is None:
            key = ""
        else:
            key = str(column).strip()
    
        return key or "value"

In [7]:
# =============================================================================
# 재무제표 정보 수집 함수
# =============================================================================

def _get_finance(
    stock: str,
    url: str,
    debug: bool = False
) -> Dict[str, List[Dict]]:
    """
    재무제표 테이블 수집
    재무제표 3종(포괄손익계산서, 재무상태표, 현금흐름표)을 멀티인덱스 DataFrame으로 구조화
    
    Args:
        stock: 종목 코드
        url: URL 템플릿 ('{stock}' 포함)
        debug: 디버그 모드
    
    Returns:
        {테이블명: 레코드_리스트} 딕셔너리
    """

    # 1. 파서 생성
    parser = FnGuideFinance(debug=debug)

    # 2. 페이지 가져오기
    response_text = parser.fetch(stock, url)
    #print(response_text, type(response_text))
    if not response_text:
        return {title: [] for title in parser.finance_table_titles}
    
    # 3. 재무제표 테이블 파싱
    result_dict = parser.parse(response_text)

    return result_dict

In [8]:
finance_url = urls.get("재무제표")

# 재무제표 수집 실행
finance_data = _get_finance(
    stock=stock,
    url=finance_url,
    debug=True  # 디버그 모드
)

INFO:__main__:Main URL: https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930
INFO:__main__:
포괄손익계산서 데이터 수집 중...
INFO:__main__:'포괄손익계산서' 테이블 발견
INFO:__main__:기간 데이터 (헤더 7개 중 6개): ['2022/12', '2023/12', '2024/12', '2025/09', '전년동기', '전년동기(%)']
INFO:__main__:발견된 행 수: 79
INFO:__main__:처리 중: 20/79 행
INFO:__main__:처리 중: 40/79 행
INFO:__main__:처리 중: 60/79 행
INFO:__main__:처리 완료: 79/79 행
INFO:__main__:완료! DataFrame shape: (6, 79)
INFO:__main__:
재무상태표 데이터 수집 중...
INFO:__main__:'재무상태표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 66
INFO:__main__:처리 중: 20/66 행
INFO:__main__:처리 중: 40/66 행
INFO:__main__:처리 중: 60/66 행
INFO:__main__:처리 완료: 66/66 행
INFO:__main__:완료! DataFrame shape: (4, 66)
INFO:__main__:
현금흐름표 데이터 수집 중...
INFO:__main__:'현금흐름표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 158
INFO:__main__:처리 중: 20/158 행
INFO:__main__:처리 중: 40/158 행
INFO:__

In [9]:
finance_data

{'포괄손익계산서': [{'period': '2022/12',
   '매출액': '3,022,314',
   '매출원가': '1,900,418',
   '매출총이익': '1,121,896',
   '판매비와관리비 / 판매비와관리비계산에 참여한 계정 펼치기': '688,130',
   '판매비와관리비 / 인건비': '80,937',
   '판매비와관리비 / 유무형자산상각비': '22,391',
   '판매비와관리비 / 연구개발비': '249,192',
   '판매비와관리비 / 광고선전비': '61,130',
   '판매비와관리비 / 판매비': '139,969',
   '판매비와관리비 / 관리비': '74,579',
   '판매비와관리비 / 기타원가성비용': '',
   '판매비와관리비 / 기타': '59,932',
   '판매비와관리비 / 영업이익': '433,766',
   '판매비와관리비 / 영업이익(발표기준)': '433,766',
   '금융수익 / 금융수익계산에 참여한 계정 펼치기': '208,290',
   '금융수익 / 이자수익': '27,205',
   '금융수익 / 배당금수익': '',
   '금융수익 / 외환이익': '165,379',
   '금융수익 / 대손충당금환입액': '',
   '금융수익 / 매출채권처분이익': '',
   '금융수익 / 당기손익-공정가치측정\xa0금융자산관련이익': '',
   '금융수익 / 금융자산처분이익': '',
   '금융수익 / 금융자산평가이익': '',
   '금융수익 / 금융자산손상차손환입': '',
   '금융수익 / 파생상품이익': '15,707',
   '금융수익 / 기타금융수익': '',
   '금융원가 / 금융원가계산에 참여한 계정 펼치기': '190,277',
   '금융원가 / 이자비용': '7,630',
   '금융원가 / 외환손실': '168,097',
   '금융원가 / 대손상각비': '',
   '금융원가 / 당기손익-공정가치측정\xa0금융자산관련손실': '',
   '금융원가 / 매출

## Snapshot | 기업정보 | Company Guide

Requests로 HTML을 가져온 후 pandas.read_html()로 파싱<br/>
DataFrame의 한글 컬럼명을 영문으로 번역

- 3개월 | 1년 | 3년
- 시세현황 [YYYY/MM/DD]
- 실적이슈
- 운용사별 보유 현황
- 주주현황
- 주주구분 현황
- 신용등급현황 기업어음 (CP)
- 신용등급현황 회사채(Bond)
- 투자의견 컨센서스
- 투자의견 및 목표주가
- 투자의견 분포
- Business Summary
- 업종 비교
  - EPS
  - PER
  - EV/EBITDA
  - ROE
  - 배당수익률
- Band Chart
  - PER Band
  - PBR Band
- 주가 및 수급현황
  - 대차잔고비중
  - 차입공매도비중
- Financial Highlight [연결|전체] [전체|연간|분기]

In [10]:
# =============================================================================
# 메인(Snapshot) 분석 클래스
# =============================================================================

class FnGuideMain:
    """
    FnGuide에서 메인(Snapshot) 정보 추출
    한글 컬럼명을 영문으로 번역하는 헬퍼 클래스

    테이블의 한글 헤더를 미리 정의된 영문 이름으로 변환하여
    데이터 분석 시 일관성 있는 컬럼명 사용 가능
    """

    # 정적 HTML에서 가져올 테이블 (원본 utils/fnguide.py)
    _main_table_selectors = [
        ("market_conditions", 0),
        ("earning_issue", 1),
        ("holdings_status", 2),
        ("governance", 3),
        ("shareholders", 4),
        ("bond_rating", 6),
        ("analysis", 7),
        ("industry_comparison", 8),
        ("financialhighlight_annual", 11),
        ("financialhighlight_netquarter", 12),
    ]

    # 한글 → 영문 컬럼명 매핑 딕셔너리
    _COLUMN_MAP = {
        "잠정실적발표예정일": "tentative_results_announcement_date",
        "예상실적(영업이익, 억원)": "expected_operating_profit_billion_krw",
        "3개월전예상실적대비(%)": "expected_operating_profit_vs_3m_pct",
        "전년동기대비(%)": "year_over_year_pct",
        "운용사명": "asset_manager_name",
        "보유수량": "shares_held",
        "시가평가액": "market_value_krw",
        "상장주식수내비중": "share_of_outstanding_shares_pct",
        "운용사내비중": "manager_portfolio_ratio_pct",
        "항목": "item",
        "보통주": "common_shares",
        "지분율": "ownership_ratio_pct",
        "최종변동일": "last_change_date",
        "주주구분": "shareholder_category",
        "대표주주수": "major_shareholder_count",
        "투자의견": "investment_opinion",
        "목표주가": "target_price",
        "추정기관수": "estimate_institution_count",
        "구분": "category",
        "코스피 전기·전자": "kospi_electronics",
        "헤더": "header",
        "헤더.1": "header_1",
        "IFRS(연결)": "ifrs_consolidated",
        "IFRS(별도)": "ifrs_individual",
    }

    def __init__(self, debug: bool = False):
        """
        Args:
            debug: 디버그 모드
        """
        self.debug = debug

    def fetch(
        self,
        stock: str,
        url: str) -> str:
        """
        URL 정보를 가져오기

        Args:
            stock (str): 종목 코드
            url (str): URL 템플릿 ('{stock}' 포함)

        Returns:
            str: 웹 페이지 텍스트
        """
        # 페이지 가져오기
        self.stock = stock
        
        url = url.format(stock=stock, timeout=30)
        logger.info(f"Main URL: {url}")
        
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            logger.error(f"페이지 요청 실패: {e}")
            return None

    def parse(
        self,
        html_text: str,
        *,
        stock: str | None = None,
    ) -> Dict:
        """
        기업 정보 | Snapshot 정보 추출 후 주요 키를 영어로 변환
        requests로 HTML을 가져온 후 pandas.read_html()로 파싱

        Args:
            html_text (str): HTML text
            stock: 종목 코드 (회사명을 "company"로 치환하기 위해 사용)

        Returns:
            Dict: 컬럼명이 번역된 DataFrame
        """

        frames = pd.read_html(StringIO(html_text))
        
        datasets = {}
        stock = stock if stock else self.stock
        
        for name, index in self._main_table_selectors:
            if index < len(frames):
                frame = frames[index]
                
                # 한글 컬럼명을 영문으로 번역
                frame = self._translate(
                    frame,
                    stock=stock,
                )
                datasets[name] = frame.to_dict(orient="records")
            else:
                logger.error(f"경고: '{name}'에 해당하는 테이블(index {index})을 찾지 못해 빈 데이터로 저장합니다.")
                datasets[name] = []

        return datasets

    def _translate(
        self,
        frame: pd.DataFrame,
        *,
        stock: str | None = None,
    ) -> pd.DataFrame:
        """
        DataFrame의 한글 컬럼명을 영문으로 번역

        단일 인덱스와 멀티 인덱스 모두 지원

        Args:
            frame: 번역할 DataFrame
            stock: 종목 코드 (회사명을 "company"로 치환하기 위해 사용)

        Returns:
            pd.DataFrame: 컬럼명이 번역된 DataFrame
        """
        if frame.empty:
            return frame

        # 종목 코드로부터 회사명 조회
        company_name = None
        if stock:
            try:
                from utils.companydict import companydict
                company_name = companydict.get_company_by_code(stock)
                if company_name:
                    company_name = self._normalize(company_name)
            except (ImportError, ModuleNotFoundError):
                try:
                    from ..companydict import companydict
                    company_name = companydict.get_company_by_code(stock)
                    if company_name:
                        company_name = self._normalize(company_name)
                except (ImportError, ModuleNotFoundError):
                    pass

        translated = frame.copy()

        # 멀티인덱스 처리
        if isinstance(translated.columns, pd.MultiIndex):
            new_columns = []
            for column in translated.columns:
                # 각 레벨별로 번역
                new_columns.append(
                    tuple(self._translate_token(level, company_name) for level in column)
                )
            translated.columns = pd.MultiIndex.from_tuples(
                new_columns, names=translated.columns.names
            )
        # 단일 인덱스 처리
        else:
            translated.rename(
                columns=lambda label: self._translate_token(label, company_name),
                inplace=True,
            )

        return translated

    def _translate_token(self, label: str, company_name: str | None) -> str:
        """
        개별 컬럼명 토큰 번역

        Args:
            label: 원본 컬럼명
            company_name: 회사명 (해당 컬럼을 "company"로 치환)

        Returns:
            str: 번역된 컬럼명
        """
        if not isinstance(label, str):
            return label

        normalized = self._normalize(label)

        # 회사명이면 "company"로 통일
        if company_name and normalized == company_name:
            return "company"

        # 매핑 딕셔너리에서 찾기 (없으면 원본 그대로)
        return self._COLUMN_MAP.get(normalized, normalized)

    @property
    @staticmethod
    def main_table_selectors(self) -> List[Tuple[str, int]]:
        """
        Snapshot HTML에서 가져올 테이블 선택자

        Returns:
            List[Tuple[str, int]]: (테이블명, 인덱스) 튜플 리스트
        """
        return self._main_table_selectors

    @staticmethod
    def _normalize(value: str) -> str:
        """
        문자열 정규화 (공백 문자 제거 및 trim)

        Args:
            value: 원본 문자열

        Returns:
            str: 정규화된 문자열
        """
        return value.replace("\xa0", " ").strip()

In [11]:
# =============================================================================
# 메인(Snapshot) 정보 수집 함수
# =============================================================================

def _get_snapshot(
    stock: str,
    url: str,
    debug: bool = False
    ) -> dict[str, list[dict]]:
    """
    기업 메인(Snapshot) 정보 수집
    시장 상황, 지배구조, 주주 현황 등의 기업 데이터를 수집

    Returns:
        dict[str, list[dict]]: 테이블명을 키로 하는 레코드 딕셔너리
    """
    
    company_info = FnGuideMain()
    response_text = company_info.fetch(stock, url)
    if not response_text:
        return {title: [] for title in table_titles}
    
    datasets = company_info.parse(response_text)

    return datasets

In [12]:
main_url = urls.get("메인")

snapshot_data = _get_snapshot(
    stock=stock,
    url=main_url)

INFO:__main__:Main URL: https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930


In [13]:
snapshot_data

{'market_conditions': [{0: '종가/ 전일대비/ 수익률',
   1: '100,500/ -3,000/ -2.90',
   2: '거래량',
   3: '15292277'},
  {0: '52주.최고가/ 최저가', 1: '111,100/ 51,000', 2: '거래대금(억원)', 3: '15489'},
  {0: '수익률(1M/ 3M/ 6M/ 1Y)',
   1: '+1.01/ +44.40/ +79.79/ +81.08',
   2: '외국인 지분율',
   3: '52.17'},
  {0: '시가총액(상장예정포함,억원)', 1: '6561217', 2: '베타(1년)', 3: '1.13511'},
  {0: '시가총액(보통주,억원)', 1: '5949236', 2: '액면가', 3: '100'},
  {0: nan, 1: nan, 2: nan, 3: nan},
  {0: '발행주식수(보통주/ 우선주)',
   1: '5,919,637,922/ 815,974,664',
   2: '종가(NXT)',
   3: '101500'},
  {0: '유동주식수/비율(보통주)',
   1: '4,474,065,669 / 75.58',
   2: '거래량/거래대금(NXT)(억원)',
   3: '8,117,272 / 8,264'}],
 'earning_issue': [{'확정실적(영업이익, 억원)': 121661,
   'expected_operating_profit_billion_krw': nan,
   'expected_operating_profit_vs_3m_pct': -57.53,
   'year_over_year_pct': 32.48}],
 'holdings_status': [{'asset_manager_name': '삼성자산운용',
   'shares_held': 63616.68,
   'market_value_krw': 53374.4,
   'share_of_outstanding_shares_pct': 1.07,
   'manager_portf

#### 시세현황
```
                     0                              1                  2  \
0        종가/ 전일대비/ 수익률           103,500/ +700/ +0.68                거래량   
1         52주.최고가/ 최저가                111,100/ 51,000           거래대금(억원)   
2  수익률(1M/ 3M/ 6M/ 1Y)  +1.47/ +46.60/ +92.02/ +83.84            외국인 지분율   
3      시가총액(상장예정포함,억원)                        6755126             베타(1년)   
4         시가총액(보통주,억원)                        6126825                액면가   
5                  NaN                            NaN                NaN   
6      발행주식수(보통주/ 우선주)     5,919,637,922/ 815,974,664            종가(NXT)   
7        유동주식수/비율(보통주)          4,474,125,919 / 75.58  거래량/거래대금(NXT)(억원)   

                     3  
0             16453004  
1                17132  
2                52.28  
3              1.13081  
4                  100  
5                  NaN  
6               103400  
7  12,226,287 / 12,721  
```


#### 실적이슈
```
   확정실적(영업이익, 억원)  예상실적(영업이익, 억원)  3개월전예상실적대비(%)  전년동기대비(%)
0          121661             NaN         -57.53      32.48 
```


#### 운용사별 보유 현황
```
          운용사명      보유수량     시가평가액  상장주식수내비중  운용사내비중
0       삼성자산운용  63616.68  53374.40      1.07    4.11
1     미래에셋자산운용  35096.76  29446.18      0.59    2.58
2      케이비자산운용  12862.25  10791.43      0.22    2.26
3     한국투자신탁운용   7681.16   6444.49      0.13    1.41
4  엔에이치아문디자산운용   7054.55   5918.77      0.12    3.34
5     교보악사자산운용   5276.65   4427.11      0.09    3.39
6     키움투자자산운용   5160.68   4329.81      0.09    2.34
7       신한자산운용   4409.80   3699.82      0.07    1.10
8   한국투자밸류자산운용   3921.28   3289.95      0.07   12.27
9       한화자산운용   3639.64   3053.66      0.06    2.07
```


#### 주주현황
```
                               항목           보통주    지분율       최종변동일
0                   삼성생명보험(외 15인)  1.174506e+09  19.84  2025/11/06
1                          국민연금공단  4.586377e+08   7.75  2022/08/16
2  BlackRock Fund Advisors(외 15인)  3.003911e+08   5.07  2019/01/28
3                             자사주  9.182899e+07   1.55  2025/10/31
4                             NaN           NaN    NaN         NaN
5                             NaN           NaN    NaN         NaN
```


#### 주주구분 현황
```
                 주주구분  대표주주수           보통주    지분율       최종변동일
0    최대주주등 (본인+특별관계자)    1.0  1.174506e+09  19.84  2025/11/06
1  10%이상주주 (본인+특별관계자)    NaN           NaN    NaN         NaN
2   5%이상주주 (본인+특별관계자)    2.0  7.590287e+08  12.82  2022/08/16
3   임원 (5%미만 중, 임원인자)  851.0  3.312332e+06   0.06  2025/11/25
4    자기주식 (자사주+자사주신탁)    1.0  9.182899e+07   1.55  2025/10/31
5              우리사주조합    NaN           NaN    NaN         NaN 
```


#### 신용등급현황 기업어음 (CP)
```
             KIS             KR           NICE
0  관련 데이터가 없습니다.  관련 데이터가 없습니다.  관련 데이터가 없습니다.
```


#### 신용등급현황 회사채(Bond)
```
                KIS                KR              NICE
0  AAA [2004/05/25]  AAA [2004/06/17]  AAA [2004/06/01]
```


#### 투자의견 컨센서스
```
   투자의견    목표주가   EPS   PER  추정기관수
0   4.0  136615  5513  18.8     26
```


#### 업종 비교 (연결)
```
          구분        삼성전자    코스피 전기·전자        KOSPI
0       시가총액  6126825.00  13935008.00  32860539.00
1        매출액  3008709.00   6133815.00  33576415.00
2       영업이익   327260.00    627085.00   2495504.00
3     EPS(원)     4950.00     22628.92      7000.82
4        PER       10.75        11.92        11.29
5  EV/EBITDA        4.84         5.72         6.44
6        ROE        9.03         9.81         7.48
7      배당수익률        2.72         1.75         2.28
8     베타(1년)        1.23         1.22         1.00
```


#### 업종 비교 (별도)
```
          구분        삼성전자    코스피 전기·전자        KOSPI
0       시가총액  6126825.00  13935008.00  32860539.00
1        매출액  2090522.00   4176342.00  17385907.00
2       영업이익   123610.00    330067.00   1395249.00
3     EPS(원)     3472.00     17450.75      5389.62
4        PER       15.32        15.45        14.66
5  EV/EBITDA        8.35         9.22         9.01
6        ROE       10.23        10.95         7.74
7      배당수익률        2.72         1.75         2.28
8     베타(1년)        1.23         1.22         1.00
```


#### Financial Highlight (연결, 전체)
```
      IFRS(연결)      Annual                                     Net Quarter  \
      IFRS(연결)     2022/12     2023/12     2024/12  2025/12(E)     2025/03   
0          매출액  3022314.00  2589355.00  3008709.00  3264438.00   791405.00   
1         영업이익   433766.00    65670.00   327260.00   382329.00    66853.00   
2   영업이익(발표기준)   433766.00    65670.00   327260.00         NaN    66853.00   
3        당기순이익   556541.00   154871.00   344514.00   380456.00    82229.00   
4      지배주주순이익   547300.00   144734.00   336214.00   371767.00    80284.00   
5     비지배주주순이익     9241.00    10137.00     8300.00         NaN     1945.00   
6         자산총계  4484245.00  4559060.00  5145319.00  5417265.00  5163767.00   
7         부채총계   936749.00   922281.00  1123399.00  1147195.00  1097625.00   
8         자본총계  3547496.00  3636779.00  4021921.00  4270070.00  4066143.00   
9       지배주주지분  3451861.00  3532338.00  3916876.00  4165171.00  3958611.00   
10     비지배주주지분    95635.00   104441.00   105045.00   104899.00   107532.00   
11         자본금     8975.00     8975.00     8975.00     8965.00     8975.00   
12        부채비율       26.41       25.36       27.93       26.87       26.99   
13         유보율    38360.25    39256.91    43743.26         NaN    44197.88   
14       영업이익률       14.35        2.54       10.88       11.71        8.45   
15    지배주주순이익률       18.11        5.59       11.17       11.39       10.14   
16         ROA       12.72        3.43        7.10        7.20        6.38   
17         ROE       17.07        4.14        9.03        9.20        8.16   
18      EPS(원)     8057.00     2131.00     4950.00     5513.00     1186.00   
19      BPS(원)    50817.00    52002.00    57930.00    62107.00    59027.00   
20      DPS(원)     1444.00     1444.00     1446.00     1521.00      365.00   
21         PER        6.86       36.84       10.75       18.77         NaN   
22         PBR        1.09        1.51        0.92        1.67        0.98   
23       발행주식수  5969783.00  5969783.00  5969783.00         NaN  5919638.00   
24       배당수익률        2.61        1.84        2.72         NaN        0.63   

                                       
       2025/06     2025/09 2025/12(E)  
0    745663.00   860617.00  868725.00  
1     46761.00   121661.00  148045.00  
2     46761.00   121661.00        NaN  
3     51164.00   122257.00  126960.00  
4     49340.00   120065.00  139116.00  
5      1824.00     2193.00        NaN  
6   5048752.00  5236596.00        NaN  
7   1053132.00  1101581.00        NaN  
8   3995620.00  4135015.00        NaN  
9   3886941.00  4020011.00        NaN  
10   108679.00   115004.00        NaN  
11     8975.00     8975.00        NaN  
12       26.36       26.64        NaN  
13    43513.36    45422.38        NaN  
14        6.27       14.14      17.04  
15        6.62       13.95      16.01  
16        4.01        9.51       9.70  
17        5.03       12.15      13.84  
18      733.00     1783.00    2063.00  
19    58114.00    60658.00        NaN  
20      367.00      370.00        NaN  
21         NaN         NaN        NaN  
22        1.03        1.38        NaN  
23  5919638.00  5919638.00        NaN  
24        0.61        0.44        NaN  
```


#### Financial Highlight (연결, 연간)
```
      IFRS(연결)      Annual                                                  \
      IFRS(연결)     2020/12     2021/12     2022/12     2023/12     2024/12   
0          매출액  2368070.00  2796048.00  3022314.00  2589355.00  3008709.00   
1         영업이익   359939.00   516339.00   433766.00    65670.00   327260.00   
2   영업이익(발표기준)   359939.00   516339.00   433766.00    65670.00   327260.00   
3        당기순이익   264078.00   399075.00   556541.00   154871.00   344514.00   
4      지배주주순이익   260908.00   392438.00   547300.00   144734.00   336214.00   
5     비지배주주순이익     3170.00     6637.00     9241.00    10137.00     8300.00   
6         자산총계  3782357.00  4266212.00  4484245.00  4559060.00  5145319.00   
7         부채총계  1022877.00  1217212.00   936749.00   922281.00  1123399.00   
8         자본총계  2759480.00  3048999.00  3547496.00  3636779.00  4021921.00   
9       지배주주지분  2676703.00  2962377.00  3451861.00  3532338.00  3916876.00   
10     비지배주주지분    82777.00    86622.00    95635.00   104441.00   105045.00   
11         자본금     8975.00     8975.00     8975.00     8975.00     8975.00   
12        부채비율       37.07       39.92       26.41       25.36       27.93   
13         유보율    29723.53    32906.47    38360.25    39256.91    43743.26   
14       영업이익률       15.20       18.47       14.35        2.54       10.88   
15    지배주주순이익률       11.02       14.04       18.11        5.59       11.17   
16         ROA        7.23        9.92       12.72        3.43        7.10   
17         ROE        9.99       13.92       17.07        4.14        9.03   
18      EPS(원)     3841.00     5777.00     8057.00     2131.00     4950.00   
19      BPS(원)    39406.00    43611.00    50817.00    52002.00    57930.00   
20      DPS(원)     2994.00     1444.00     1444.00     1444.00     1446.00   
21         PER       21.09       13.55        6.86       36.84       10.75   
22         PBR        2.06        1.80        1.09        1.51        0.92   
23       발행주식수  5969783.00  5969783.00  5969783.00  5969783.00  5969783.00   
24       배당수익률        3.70        1.84        2.61        1.84        2.72   

                                        
    2025/12(E)  2026/12(E)  2027/12(E)  
0   3264438.00  3843059.00  4109003.00  
1    382329.00   800114.00   838110.00  
2          NaN         NaN         NaN  
3    380456.00   691879.00   737735.00  
4    371767.00   685954.00   723747.00  
5          NaN         NaN         NaN  
6   5417265.00  6107563.00  6856039.00  
7   1147195.00  1222698.00  1258347.00  
8   4270070.00  4884865.00  5597800.00  
9   4165171.00  4778756.00  5475013.00  
10   104899.00   106109.00   122787.00  
11     8965.00     8965.00     8962.00  
12       26.87       25.03       22.48  
13         NaN         NaN         NaN  
14       11.71       20.82       20.40  
15       11.39       17.85       17.61  
16        7.20       12.01       11.38  
17        9.20       15.34       14.12  
18     5513.00    10184.00    10745.00  
19    62107.00    71217.00    81554.00  
20     1521.00     1583.00     1464.00  
21       18.77       10.16        9.63  
22        1.67        1.45        1.27  
23         NaN         NaN         NaN  
24         NaN         NaN         NaN  
```


#### Financial Highlight (연결, 분기)
```
      IFRS(연결) Net Quarter                                                  \
      IFRS(연결)     2024/09     2024/12     2025/03     2025/06     2025/09   
0          매출액   790987.00   757883.00   791405.00   745663.00   860617.00   
1         영업이익    91834.00    64927.00    66853.00    46761.00   121661.00   
2   영업이익(발표기준)    91834.00    64927.00    66853.00    46761.00   121661.00   
3        당기순이익   101009.00    77544.00    82229.00    51164.00   122257.00   
4      지배주주순이익    97815.00    75761.00    80284.00    49340.00   120065.00   
5     비지배주주순이익     3194.00     1783.00     1945.00     1824.00     2193.00   
6         자산총계  4913073.00  5145319.00  5163767.00  5048752.00  5236596.00   
7         부채총계  1050260.00  1123399.00  1097625.00  1053132.00  1101581.00   
8         자본총계  3862814.00  4021921.00  4066143.00  3995620.00  4135015.00   
9       지배주주지분  3761522.00  3916876.00  3958611.00  3886941.00  4020011.00   
10     비지배주주지분   101292.00   105045.00   107532.00   108679.00   115004.00   
11         자본금     8975.00     8975.00     8975.00     8975.00     8975.00   
12        부채비율       27.19       27.93       26.99       26.36       26.64   
13         유보율    41810.45    43743.26    44197.88    43513.36    45422.38   
14       영업이익률       11.61        8.57        8.45        6.27       14.14   
15    지배주주순이익률       12.37       10.00       10.14        6.62       13.95   
16         ROA        8.27        6.17        6.38        4.01        9.51   
17         ROE       10.44        7.89        8.16        5.03       12.15   
18      EPS(원)     1440.00     1115.00     1186.00      733.00     1783.00   
19      BPS(원)    55376.00    57930.00    59027.00    58114.00    60658.00   
20      DPS(원)      361.00      363.00      365.00      367.00      370.00   
21         PER         NaN         NaN         NaN         NaN         NaN   
22         PBR        1.11        0.92        0.98        1.03        1.38   
23       발행주식수  5969783.00  5969783.00  5919638.00  5919638.00  5919638.00   
24       배당수익률        0.59        0.68        0.63        0.61        0.44   

                                     
   2025/12(E) 2026/03(E) 2026/06(E)  
0   868725.00  931259.00  932299.00  
1   148045.00  176177.00  199553.00  
2         NaN        NaN        NaN  
3   126960.00  153805.00  164448.00  
4   139116.00  153026.00  162615.00  
5         NaN        NaN        NaN  
6         NaN        NaN        NaN  
7         NaN        NaN        NaN  
8         NaN        NaN        NaN  
9         NaN        NaN        NaN  
10        NaN        NaN        NaN  
11        NaN        NaN        NaN  
12        NaN        NaN        NaN  
13        NaN        NaN        NaN  
14      17.04      18.92      21.40  
15      16.01      16.43      17.44  
16       9.70        NaN        NaN  
17      13.84        NaN        NaN  
18    2063.00    2272.00    2414.00  
19        NaN        NaN        NaN  
20        NaN        NaN        NaN  
21        NaN        NaN        NaN  
22        NaN        NaN        NaN  
23        NaN        NaN        NaN  
24        NaN        NaN        NaN  
```


#### Financial Highlight (별도, 전체)
```
      IFRS(별도)      Annual                                    Net Quarter  \
      IFRS(별도)     2022/12     2023/12     2024/12 2025/12(E)     2025/03   
0          매출액  2118675.00  1703741.00  2090522.00        NaN   555349.00   
1         영업이익   253193.00  -115263.00   123610.00        NaN    14693.00   
2   영업이익(발표기준)   253193.00  -115263.00   123610.00        NaN    14693.00   
3        당기순이익   254188.00   253971.00   235826.00        NaN    63872.00   
4         자산총계  2600838.00  2968573.00  3249661.00        NaN  3225380.00   
5         부채총계   506676.00   720695.00   885695.00        NaN   846823.00   
6         자본총계  2094162.00  2247878.00  2363967.00        NaN  2378557.00   
7          자본금     8975.00     8975.00     8975.00        NaN     8975.00   
8         부채비율       24.19       32.06       37.47        NaN       35.60   
9          유보율    23232.92    24945.60    26440.92        NaN    26593.10   
10       영업이익률       11.95       -6.77        5.91        NaN        2.65   
11        순이익률       12.00       14.91       11.28        NaN       11.50   
12         ROA        9.94        9.12        7.58        NaN        7.89   
13         ROE       12.63       11.70       10.23        NaN       10.77   
14      EPS(원)     3742.00     3739.00     3472.00        NaN      944.00   
15      BPS(원)    30830.00    33093.00    35068.00        NaN    35568.00   
16      DPS(원)     1444.00     1444.00     1446.00        NaN      365.00   
17         PER       14.78       21.00       15.32        NaN         NaN   
18         PBR        1.79        2.37        1.52        NaN        1.63   
19       발행주식수  5969783.00  5969783.00  5969783.00        NaN  5919638.00   
20       배당수익률        2.61        1.84        2.72        NaN        0.63   

                                       
       2025/06     2025/09 2025/12(E)  
0    547654.00   627991.00        NaN  
1     11908.00    79411.00        NaN  
2     11908.00    79411.00        NaN  
3     27520.00   106686.00        NaN  
4   3220174.00  3254934.00        NaN  
5    846719.00   831082.00        NaN  
6   2373455.00  2423852.00        NaN  
7      8975.00     8975.00        NaN  
8        35.67       34.29        NaN  
9     26650.27    27638.16        NaN  
10        2.17       12.65        NaN  
11        5.03       16.99        NaN  
12        3.42       13.18        NaN  
13        4.63       17.79        NaN  
14      409.00     1584.00        NaN  
15    35644.00    36961.00        NaN  
16      367.00      370.00        NaN  
17         NaN         NaN        NaN  
18        1.68        2.27        NaN  
19  5919638.00  5919638.00        NaN  
20        0.61        0.44        NaN  
```


#### Financial Highlight (별도, 연간)
```
      IFRS(별도)      Annual                                                  \
      IFRS(별도)     2020/12     2021/12     2022/12     2023/12     2024/12   
0          매출액  1663112.00  1997447.00  2118675.00  1703741.00  2090522.00   
1         영업이익   205190.00   319932.00   253193.00  -115263.00   123610.00   
2   영업이익(발표기준)   205190.00   319932.00   253193.00  -115263.00   123610.00   
3        당기순이익   156150.00   309710.00   254188.00   253971.00   235826.00   
4         자산총계  2296644.00  2511122.00  2600838.00  2968573.00  3249661.00   
5         부채총계   463477.00   579185.00   506676.00   720695.00   885695.00   
6         자본총계  1833167.00  1931937.00  2094162.00  2247878.00  2363967.00   
7          자본금     8975.00     8975.00     8975.00     8975.00     8975.00   
8         부채비율       25.28       29.98       24.19       32.06       37.47   
9          유보율    20324.94    21425.43    23232.92    24945.60    26440.92   
10       영업이익률       12.34       16.02       11.95       -6.77        5.91   
11        순이익률        9.39       15.51       12.00       14.91       11.28   
12         ROA        7.00       12.88        9.94        9.12        7.58   
13         ROE        8.65       16.45       12.63       11.70       10.23   
14      EPS(원)     2299.00     4559.00     3742.00     3739.00     3472.00   
15      BPS(원)    26987.00    28442.00    30830.00    33093.00    35068.00   
16      DPS(원)     2994.00     1444.00     1444.00     1444.00     1446.00   
17         PER       35.24       17.17       14.78       21.00       15.32   
18         PBR        3.00        2.75        1.79        2.37        1.52   
19       발행주식수  5969783.00  5969783.00  5969783.00  5969783.00  5969783.00   
20       배당수익률        3.70        1.84        2.61        1.84        2.72   

                                     
   2025/12(E) 2026/12(E) 2027/12(E)  
0         NaN        NaN        NaN  
1         NaN        NaN        NaN  
2         NaN        NaN        NaN  
3         NaN        NaN        NaN  
4         NaN        NaN        NaN  
5         NaN        NaN        NaN  
6         NaN        NaN        NaN  
7         NaN        NaN        NaN  
8         NaN        NaN        NaN  
9         NaN        NaN        NaN  
10        NaN        NaN        NaN  
11        NaN        NaN        NaN  
12        NaN        NaN        NaN  
13        NaN        NaN        NaN  
14        NaN        NaN        NaN  
15        NaN        NaN        NaN  
16        NaN        NaN        NaN  
17        NaN        NaN        NaN  
18        NaN        NaN        NaN  
19        NaN        NaN        NaN  
20        NaN        NaN        NaN  
```


#### Financial Highlight (별도, 분기)
```
      IFRS(별도) Net Quarter                                                  \
      IFRS(별도)     2024/09     2024/12     2025/03     2025/06     2025/09   
0          매출액   558289.00   482496.00   555349.00   547654.00   627991.00   
1         영업이익    45049.00   -13738.00    14693.00    11908.00    79411.00   
2   영업이익(발표기준)    45049.00   -13738.00    14693.00    11908.00    79411.00   
3        당기순이익    56946.00    10521.00    63872.00    27520.00   106686.00   
4         자산총계  3233828.00  3249661.00  3225380.00  3220174.00  3254934.00   
5         부채총계   833108.00   885695.00   846823.00   846719.00   831082.00   
6         자본총계  2400720.00  2363967.00  2378557.00  2373455.00  2423852.00   
7          자본금     8975.00     8975.00     8975.00     8975.00     8975.00   
8         부채비율       34.70       37.47       35.60       35.67       34.29   
9          유보율    26648.55    26440.92    26593.10    26650.27    27638.16   
10       영업이익률        8.07       -2.85        2.65        2.17       12.65   
11        순이익률       10.20        2.18       11.50        5.03       16.99   
12         ROA        7.11        1.30        7.89        3.42       13.18   
13         ROE        9.55        1.77       10.77        4.63       17.79   
14      EPS(원)      838.00      155.00      944.00      409.00     1584.00   
15      BPS(원)    35343.00    35068.00    35568.00    35644.00    36961.00   
16      DPS(원)      361.00      363.00      365.00      367.00      370.00   
17         PER         NaN         NaN         NaN         NaN         NaN   
18         PBR        1.74        1.52        1.63        1.68        2.27   
19       발행주식수  5969783.00  5969783.00  5919638.00  5919638.00  5919638.00   
20       배당수익률        0.59        0.68        0.63        0.61        0.44   

                                     
   2025/12(E) 2026/03(E) 2026/06(E)  
0         NaN        NaN        NaN  
1         NaN        NaN        NaN  
2         NaN        NaN        NaN  
3         NaN        NaN        NaN  
4         NaN        NaN        NaN  
5         NaN        NaN        NaN  
6         NaN        NaN        NaN  
7         NaN        NaN        NaN  
8         NaN        NaN        NaN  
9         NaN        NaN        NaN  
10        NaN        NaN        NaN  
11        NaN        NaN        NaN  
12        NaN        NaN        NaN  
13        NaN        NaN        NaN  
14        NaN        NaN        NaN  
15        NaN        NaN        NaN  
16        NaN        NaN        NaN  
17        NaN        NaN        NaN  
18        NaN        NaN        NaN  
19        NaN        NaN        NaN  
20        NaN        NaN        NaN  
```


#### 
```
   헤더 헤더.1
0  내용   내용
1  내용   내용
2  내용   내용
3  내용   내용
```

In [14]:
class FnGuideCrawler:
    """FnGuide 데이터 크롤러 - 메인(Snapshot) / 재무제표 테이블 수집 + GCS 저장"""

    # 재무제표 테이블
    finance_table_titles = ["포괄손익계산서", "재무상태표", "현금흐름표"]

    # 파티션 폴더명 접두사 (GCS 저장 경로용)
    quarter_prefix = "quarter="

    def __init__(self, stock: str = "005930", bucket_name: str = "sayouzone-ai-stocks"):
        """
        FnGuide 크롤러 초기화

        Args:
            stock: 종목 코드 (6자리, 예: "005930")
            bucket_name: GCS 버킷 이름
        """
        self.stock = stock

        #Snapshot(메인)
        self.main_url = urls.get("메인")
        #재무제표
        self.finance_url = urls.get("재무제표")

        # GCS Manager 초기화 (지연 초기화 패턴)
        self.bucket_name = bucket_name
        self._gcs = None
        self._gcs_initialized = False

    @staticmethod
    def _flatten_column_key(column: Any) -> str:
        """
        멀티인덱스 컬럼 키를 JSON 직렬화가 가능한 단일 문자열로 변환한다.
        """
        if isinstance(column, tuple):
            parts = [str(part).strip() for part in column if part not in (None, "")]
            key = " / ".join(parts)
        elif column is None:
            key = ""
        else:
            key = str(column).strip()

        return key or "value"

    def _dataframe_to_records(self, frame: pd.DataFrame) -> list[dict[str, Any]]:
        """
        DataFrame을 JSON 직렬화를 위한 레코드 리스트로 변환한다.

        멀티인덱스 컬럼은 " / "로 결합된 단일 키로 변환하고,
        인덱스(기간)는 'period' 필드로 포함한다.
        """
        if frame.empty:
            return []

        flattened_columns = [self._flatten_column_key(col) for col in frame.columns]
        records: list[dict[str, Any]] = []

        for index_label, row in frame.iterrows():
            record: dict[str, Any] = {"period": str(index_label)}
            for key, value in zip(flattened_columns, row.tolist()):
                if key in record:
                    suffix = 2
                    new_key = f"{key}_{suffix}"
                    while new_key in record:
                        suffix += 1
                        new_key = f"{key}_{suffix}"
                    record[new_key] = value
                else:
                    record[key] = value
            records.append(record)

        return records

    def fundamentals(
        self,
        stock: str | None = None,
        *,
        use_cache: bool = False,
        overwrite: bool = True,
    ) -> dict[str, str | None]:
        """
        기존 시스템 호환성을 위한 별칭 메서드

        routers/fundamentals.py에서 호출하는 메서드명과 일치시키기 위한
        래퍼 메서드입니다. 내부적으로 get_all_fundamentals()를 호출합니다.

        Args:
            stock: 종목 코드 (None이면 초기화 시 설정한 종목 사용)
            use_cache: True일 경우 GCS에서 캐시된 데이터 조회 시도
            overwrite: False일 경우 기존 파일이 있으면 덮어쓰지 않음

        Returns:
            dict: {
                "ticker": str,
                "country": str,
                "balance_sheet": str | None,      # JSON 문자열
                "income_statement": str | None,   # JSON 문자열
                "cash_flow": str | None           # JSON 문자열
            }
        """
        return self.get_all_fundamentals(
            stock=stock,
            use_cache=use_cache,
            overwrite=overwrite,
        )

    def get_all_fundamentals(
        self,
        stock: str | None = None,
        *,
        use_cache: bool = False,
        overwrite: bool = True,
    ) -> dict[str, str | None]:
        """
        재무제표 3종을 yfinance와 동일한 스키마로 반환 (GCS 캐싱/저장 포함)

        Args:
            stock: 종목 코드 (None이면 초기화 시 설정한 종목 사용)
            use_cache: True일 경우 GCS에서 캐시된 데이터 조회 시도
            overwrite: False일 경우 기존 파일이 있으면 덮어쓰지 않음

        Returns:
            dict: {
                "ticker": str,
                "country": str,
                "balance_sheet": str | None,      # JSON 문자열
                "income_statement": str | None,   # JSON 문자열
                "cash_flow": str | None           # JSON 문자열
            }
        """
        if stock:
            self.stock = stock
            self.main_url = urls.get("메인")
            self.finance_url = urls.get("재무제표")

        """
        result = self._check(stock: str)
        if result:
            return result
        """
        
        # 데이터 수집
        raw_data = {}

        # 1. Snapshot 정보 수집은 GCS에만 저장 (기존 호환성 유지)
        print(stock, self.main_url, self.finance_url)
        snapshot_data = self._get_snapshot(stock, self.main_url)
        raw_data.update(snapshot_data)

        # 2. 재무제표 수집 (Playwright로 재무제표 3종)
        finance_data = self._get_finance(stock, self.finance_url)
        raw_data.update(finance_data)

        """
        # 3. GCS 업로드용 CSV 페이로드 생성
        self._store(raw_data)
        """

        # 5. yfinance와 동일한 스키마로 재무제표 3종만 반환
        result = self._normalize_finance(finance_data)

        return result

    def _get_snapshot(self, stock, url) -> dict[str, list[dict]]:
        """
        기업 메인(Snapshot) 정보 수집

        requests로 HTML을 가져온 후 pandas.read_html()로 파싱하여
        시장 상황, 지배구조, 주주 현황 등의 정적 데이터를 수집

        Returns:
            dict[str, list[dict]]: 테이블명을 키로 하는 레코드 딕셔너리
        """
        company_info = FnGuideMain()
        response_text = company_info.fetch(stock, url)
        if not response_text:
            return {title: [] for title in table_titles}
        
        datasets = company_info.parse(response_text)
    
        return datasets


    def _get_finance(self, stock, url) -> dict[str, list[dict]]:
        """
        재무제표 테이블 수집 (requests + BeautifulSoup 사용)

        재무제표 3종(포괄손익계산서, 재무상태표, 현금흐름표)을
        requests와 BeautifulSoup으로 크롤링하여 멀티인덱스 DataFrame으로 구조화

        Returns:
            dict[str, list[dict]]: 테이블명을 키로 하는 레코드 딕셔너리
        """
        # 1. 파서 생성
        parser = FnGuideFinance()
    
        # 2. 페이지 가져오기
        response_text = parser.fetch(stock, url)
        #print(response_text, type(response_text))
        if not response_text:
            return {title: [] for title in parser.finance_table_titles}
        
        # 3. 재무제표 테이블 파싱
        result_dict = parser.parse(response_text)
    
        return result_dict

    def _check(self, stock: str):
        # GCS 폴더 구조 설정 (파티션 기반: year=YYYY/quarter=Q/)
        today = date.today()
        quarter = (today.month - 1) // 3 + 1  # 1~3월=1, 4~6월=2, 7~9월=3, 10~12월=4
        year_partition = f"year={today.year}"
        quarter_partition = f"{self.quarter_prefix}{quarter}"
        folder_name = (
            f"Fundamentals/FnGuide/{year_partition}/"
            f"{quarter_partition}/"
        )
        file_base = stock

        # 레거시 호환성을 위한 폴더 경로 설정
        misspelled_folder = self._partition_alias(folder_name)  # quater= 오타 지원
        legacy_folder = self._legacy_folder_from_current(
            folder_name, stock=self.stock, year=today.year, quarter=quarter
        )
        existing_files: dict[str, str] | None = None
        cached_data: dict[str, list[dict]] | None = None

        # 캐시 사용 로직 (GCS가 사용 가능한 경우에만)
        if use_cache and self.gcs is not None:
            # GCS에서 기존 파일 목록 수집
            existing_files = self._collect_existing_files(
                folder_name,
                misspelled_folder,
                legacy_folder,
            )
            # 캐시된 데이터 로드 시도
            cached_data = self._load_from_gcs(
                folder_name,
                file_base,
                existing_files,
                legacy_folder,
            )
            # 캐시가 있고 overwrite=False이면 새 스키마로 변환하여 반환
            if cached_data is not None and not overwrite:
                return self._convert_to_new_schema(cached_data)
        elif use_cache and self.gcs is None:
            print("⚠ GCS를 사용할 수 없어 캐시를 사용할 수 없습니다. 새로 크롤링합니다.")

        # overwrite=False이고 아직 파일 목록을 가져오지 않았다면 수집
        if existing_files is None and not overwrite and self.gcs is not None:
            existing_files = self._collect_existing_files(
                folder_name,
                misspelled_folder,
                legacy_folder,
            )

        return None

    def _store(self, raw_data):
        csv_payloads: dict[str, str] = {}
        for name, records in raw_data.items():
            if records:  # 빈 리스트가 아닐 때만 CSV 변환
                try:
                    df = pd.DataFrame(records)
                    csv_payloads[name] = df.to_csv(index=False)
                except Exception as e:
                    print(f"'{name}' CSV 변환 실패: {e}")
                    csv_payloads[name] = ""
            else:
                csv_payloads[name] = ""

        # 4. GCS에 업로드 (GCS가 사용 가능한 경우에만)
        if self.gcs is not None:
            self.upload_to_gcs(
                csv_payloads,
                folder_name=folder_name,
                file_base=file_base,
                existing_files=existing_files,
                overwrite=overwrite,
                legacy_folder=legacy_folder,
            )
        else:
            print("⚠ GCS를 사용할 수 없어 데이터를 업로드하지 않습니다.")

    def _normalize_finance(self, finance_data: dict) -> dict[str, str | None]:
        """
        FnGuide Finance Data의 한글 키명 → 영문 키명 매핑
        
        Args:
            finance_data (dict):

        Returns:
            dict
        """
        
        # FnGuide 한글 키명 → 영문 키명 매핑
        result = {
            "ticker": self.stock,
            "country": "KR",
            "balance_sheet": None,
            "income_statement": None,
            "cash_flow": None
        }

        # 디버깅: 수집된 동적 데이터 키 확인
        print(f"[DEBUG] 수집된 동적 데이터 키: {list(finance_data.keys())}")
        for key, value in finance_data.items():
            if isinstance(value, list):
                print(f"[DEBUG] {key}: {len(value)}개 레코드")
            else:
                print(f"[DEBUG] {key}: {type(value)}")

        # 재무상태표 (포괄손익계산서 → income_statement)
        print(finance_data["포괄손익계산서"])
        if "포괄손익계산서" in finance_data and finance_data["포괄손익계산서"]:
            result["income_statement"] = json.dumps(
                finance_data["포괄손익계산서"],
                ensure_ascii=False
            )
            print(f"[DEBUG] income_statement 변환 완료: {len(result['income_statement'])} bytes")
        else:
            print(f"[DEBUG] 포괄손익계산서 데이터 없음")

        # 재무상태표
        print(finance_data["재무상태표"])
        if "재무상태표" in finance_data and finance_data["재무상태표"]:
            result["balance_sheet"] = json.dumps(
                finance_data["재무상태표"],
                ensure_ascii=False
            )
            print(f"[DEBUG] balance_sheet 변환 완료: {len(result['balance_sheet'])} bytes")
        else:
            print(f"[DEBUG] 재무상태표 데이터 없음")

        # 현금흐름표
        #print(finance_data["현금흐름표"])
        if "현금흐름표" in finance_data and finance_data["현금흐름표"]:
            result["cash_flow"] = json.dumps(
                finance_data["현금흐름표"],
                ensure_ascii=False
            )
            print(f"[DEBUG] cash_flow 변환 완료: {len(result['cash_flow'])} bytes")
        else:
            print(f"[DEBUG] 현금흐름표 데이터 없음")

        return result

In [15]:
use_cache = True
stock = "005930" # 삼성전자
crawler = FnGuideCrawler(stock=stock)
data = crawler.fundamentals(stock=stock, use_cache=use_cache)

INFO:__main__:Main URL: https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930


005930 https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{stock} https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{stock}


INFO:__main__:Main URL: https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930
INFO:__main__:
포괄손익계산서 데이터 수집 중...
INFO:__main__:'포괄손익계산서' 테이블 발견
INFO:__main__:기간 데이터 (헤더 7개 중 6개): ['2022/12', '2023/12', '2024/12', '2025/09', '전년동기', '전년동기(%)']
INFO:__main__:발견된 행 수: 79
INFO:__main__:처리 중: 20/79 행
INFO:__main__:처리 중: 40/79 행
INFO:__main__:처리 중: 60/79 행
INFO:__main__:처리 완료: 79/79 행
INFO:__main__:완료! DataFrame shape: (6, 79)
INFO:__main__:
재무상태표 데이터 수집 중...
INFO:__main__:'재무상태표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 66
INFO:__main__:처리 중: 20/66 행
INFO:__main__:처리 중: 40/66 행
INFO:__main__:처리 중: 60/66 행
INFO:__main__:처리 완료: 66/66 행
INFO:__main__:완료! DataFrame shape: (4, 66)
INFO:__main__:
현금흐름표 데이터 수집 중...
INFO:__main__:'현금흐름표' 테이블 발견
INFO:__main__:기간 데이터 (헤더 5개 중 4개): ['2022/12', '2023/12', '2024/12', '2025/09']
INFO:__main__:발견된 행 수: 158
INFO:__main__:처리 중: 20/158 행
INFO:__main__:처리 중: 40/158 행
INFO:__

[DEBUG] 수집된 동적 데이터 키: ['포괄손익계산서', '재무상태표', '현금흐름표']
[DEBUG] 포괄손익계산서: 6개 레코드
[DEBUG] 재무상태표: 4개 레코드
[DEBUG] 현금흐름표: 4개 레코드
[{'period': '2022/12', '매출액': '3,022,314', '매출원가': '1,900,418', '매출총이익': '1,121,896', '판매비와관리비 / 판매비와관리비계산에 참여한 계정 펼치기': '688,130', '판매비와관리비 / 인건비': '80,937', '판매비와관리비 / 유무형자산상각비': '22,391', '판매비와관리비 / 연구개발비': '249,192', '판매비와관리비 / 광고선전비': '61,130', '판매비와관리비 / 판매비': '139,969', '판매비와관리비 / 관리비': '74,579', '판매비와관리비 / 기타원가성비용': '', '판매비와관리비 / 기타': '59,932', '판매비와관리비 / 영업이익': '433,766', '판매비와관리비 / 영업이익(발표기준)': '433,766', '금융수익 / 금융수익계산에 참여한 계정 펼치기': '208,290', '금융수익 / 이자수익': '27,205', '금융수익 / 배당금수익': '', '금융수익 / 외환이익': '165,379', '금융수익 / 대손충당금환입액': '', '금융수익 / 매출채권처분이익': '', '금융수익 / 당기손익-공정가치측정\xa0금융자산관련이익': '', '금융수익 / 금융자산처분이익': '', '금융수익 / 금융자산평가이익': '', '금융수익 / 금융자산손상차손환입': '', '금융수익 / 파생상품이익': '15,707', '금융수익 / 기타금융수익': '', '금융원가 / 금융원가계산에 참여한 계정 펼치기': '190,277', '금융원가 / 이자비용': '7,630', '금융원가 / 외환손실': '168,097', '금융원가 / 대손상각비': '', '금융원가 / 당기손익-공정가치측정\xa0금융자산관련손실': ''

In [16]:
data

{'ticker': '005930',
 'country': 'KR',
 'balance_sheet': '[{"period": "2022/12", "자산": "4,484,245", "유동자산 / 유동자산계산에 참여한 계정 펼치기": "2,184,706", "유동자산 / 재고자산": "521,879", "유동자산 / 유동생물자산": "", "유동자산 / 유동금융자산": "655,466", "유동자산 / 매출채권및기타유동채권": "418,708", "유동자산 / 당기법인세자산": "", "유동자산 / 계약자산": "", "유동자산 / 반품(환불)자산": "", "유동자산 / 배출권": "", "유동자산 / 기타유동자산": "91,847", "유동자산 / 현금및현금성자산": "496,807", "유동자산 / 매각예정비유동자산및처분자산집단": "", "비유동자산 / 비유동자산계산에 참여한 계정 펼치기": "2,299,539", "비유동자산 / 유형자산": "1,680,454", "비유동자산 / 무형자산": "202,178", "비유동자산 / 비유동생물자산": "", "비유동자산 / 투자부동산": "", "비유동자산 / 장기금융자산": "128,025", "비유동자산 / 관계기업등지분관련투자자산": "108,939", "비유동자산 / 장기매출채권및기타비유동채권": "10,285", "비유동자산 / 이연법인세자산": "51,013", "비유동자산 / 장기당기법인세자산": "", "비유동자산 / 계약자산": "", "비유동자산 / 반품(환불)자산": "", "비유동자산 / 배출권": "", "비유동자산 / 기타비유동자산": "118,646", "비유동자산 / 기타금융업자산": "", "비유동자산 / 부채": "936,749", "유동부채 / 유동부채계산에 참여한 계정 펼치기": "783,449", "유동부채 / 단기사채": "", "유동부채 / 단기차입금": "51,473", "유동부채 / 유동성장기부채": "10,892", "유동부채 / 유동금융부채": "", "유동부채 